In [1]:
import pandas as pd
import requests
import json
import time

## OBJECTIF : CLEANER LE DATASET

In [2]:
df = pd.read_csv('products.csv', sep = ';' )
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
username      200 non-null object
ip_address    200 non-null object
product       200 non-null object
price         200 non-null object
infos         200 non-null object
dtypes: object(5)
memory usage: 7.9+ KB


In [3]:
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


### On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.

In [4]:
df['price']=df.price.str.split().str[0]

In [5]:
df['price']=pd.to_numeric(df.price)

In [6]:
def getCountry(ip):
    
    test=f"https://freegeoip.app/json/{ip}"
    resp = requests.get(test)
    data = resp.json()['country_code']
    
    time.sleep(0.2)
    return data

In [7]:
# Python program to validate an Ip addess . SOURCE= https://www.geeksforgeeks.org/python-program-to-validate-an-ip-address/
  
# re module provides support 
# for regular expressions 
import re 
  
# Make a regular expression 
# for validating an Ip-address 
regex = '''^(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)'''
      
# Define a function to validate an Ip addess 
def check(Ip):  
  
    # pass the regular expression 
    # and the string in search() method 
    if(re.search(regex, Ip)):  
        return True
          

In [8]:
# create a column valid_ip
df['valid_ip'] = df['ip_address'].apply(check)

In [9]:
# drop all the rows whose valid_ip is NA
df = df.dropna(subset = ['valid_ip'])

In [10]:
#to ensure it works, we will reduce the size of the dataframe
df = df[:100]

In [11]:
#find the country for each row based on the ip
df['country'] = df['ip_address'].apply(getCountry)

In [12]:
#create a dictionary {COUNTRY : local_CURRENCY}
url_currency= 'http://country.io/currency.json'
res= requests.get(url_currency)
dico_currency = res.json()

In [13]:
#create a column with local currency for each row, based on column 'country'
df['local_currency']=df['country'].map(dico_currency)

In [14]:
#drop all the rows whose local_currency is NA
df = df.dropna(subset = ['local_currency'])

In [17]:
#create a dictionnary of exchange rate, base currency = Eur
url_currency_euro_rate= 'https://api.exchangerate-api.com/v4/latest/EUR'
res2= requests.get(url_currency_euro_rate)
dico_converter = res2.json()['rates']

In [18]:
dico_converter

{'EUR': 1,
 'AED': 4.079015,
 'ARS': 66.028642,
 'AUD': 1.618497,
 'BGN': 1.955821,
 'BRL': 4.453402,
 'BSD': 1.110871,
 'CAD': 1.457975,
 'CHF': 1.103365,
 'CLP': 807.980283,
 'CNY': 7.844961,
 'COP': 3667.590909,
 'CZK': 25.528976,
 'DKK': 7.470666,
 'DOP': 58.553701,
 'EGP': 17.886721,
 'FJD': 2.435025,
 'GBP': 0.862861,
 'GTQ': 8.592865,
 'HKD': 8.718952,
 'HRK': 7.460829,
 'HUF': 329.56967,
 'IDR': 15701.993282,
 'ILS': 3.92213,
 'INR': 78.798349,
 'ISK': 138.049729,
 'JPY': 121.059485,
 'KRW': 1298.071593,
 'KZT': 431.481283,
 'MXN': 21.269838,
 'MYR': 4.646433,
 'NOK': 10.251419,
 'NZD': 1.747355,
 'PAB': 1.110871,
 'PEN': 3.713692,
 'PHP': 56.616652,
 'PKR': 172.040512,
 'PLN': 4.265235,
 'PYG': 7335.181818,
 'RON': 4.756323,
 'RUB': 71.011911,
 'SAR': 4.168365,
 'SEK': 10.792381,
 'SGD': 1.514143,
 'THB': 33.603543,
 'TRY': 6.366114,
 'TWD': 33.883093,
 'UAH': 27.595224,
 'USD': 1.112036,
 'UYU': 41.612687,
 'VND': 25652.631579,
 'ZAR': 16.453618}

In [19]:
#create a column with converted price from local_currency to euro
df['price_in_euro']=round(df['price']* df['local_currency'].map(dico_converter),2)

In [20]:
df

,username,ip_address,product,price,infos,valid_ip,country,local_currency,price_in_euro
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,True,US,USD,389.38
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",True,JP,JPY,114981.09
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,True,MX,MXN,2372.86
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",True,MX,MXN,1872.81
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",True,US,USD,957.74
...,...,...,...,...,...,...,...,...,...
104,bmclevie2w,136.232.122.2,"Lentils - Red, Dry",99.92,Contains mustard and sugar,True,IN,INR,7873.53
106,nlampkin2y,49.215.30.109,Tomato - Plum With Basil,990.81,"Ingredients: peanut, mustard, sugar and egg",True,TW,TWD,33571.71
107,ctowndrow2z,87.59.190.114,Bread - Pita,767.62,"May contain soja, sugar and egg",True,DK,DKK,5734.63
108,fjennemann30,223.203.79.58,Wine - Red Oakridge Merlot,372.13,"Ingredients: gluten, sugar and fish",True,CN,CNY,2919.35


### La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

In [21]:
df["infos"]=df.infos.str.replace(',',' ')
df["sugar"]=df.infos.str.get_dummies(sep=' ').sugar
df["peanut"]=df.infos.str.get_dummies(sep=' ').peanut
df["fish"] =df.infos.str.get_dummies(sep=' ').fish
df["mustard"] =df.infos.str.get_dummies(sep=' ').mustard
df["gluten"] =df.infos.str.get_dummies(sep=' ').gluten
df["egg"] =df.infos.str.get_dummies(sep=' ').egg
df["milk"] =df.infos.str.get_dummies(sep=' ').milk
df["soja"] =df.infos.str.get_dummies(sep=' ').soja


In [26]:
df = df.drop(columns="infos")

In [28]:
df.head()

,username,ip_address,product,price,valid_ip,country,local_currency,price_in_euro,sugar,peanut,fish,mustard,gluten,egg,milk,soja
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,True,US,USD,389.38,0,0,0,0,1,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,True,JP,JPY,114981.09,1,0,1,0,0,1,0,0
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,True,MX,MXN,2372.86,1,0,0,0,0,0,0,0
8,epridham8,187.129.113.105,Dried Figs,88.05,True,MX,MXN,1872.81,1,0,1,0,0,0,1,0
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,True,US,USD,957.74,1,1,1,0,0,0,1,0
